In [1]:
from pycaret.classification import *
from sklearn.model_selection import train_test_split

import pandas as pd
import os

In [2]:
ROOT_DIR = "../data"
RANDOM_STATE = 200

train_data = pd.read_csv(os.path.join(ROOT_DIR, "pca_train_.csv"))
train_data.rename(columns={'12': 'target'}, inplace=True)

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

normal_ratio = 0.5

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)

## under sampling
df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)

train_data = df_concat

# df_train, df_val = train_test_split(
#     df_concat,
#     test_size=0.3,
#     stratify=df_concat["target"],
#     random_state=RANDOM_STATE,
# )

In [3]:
setup_clf = setup(data=train_data, target="target", train_size=0.7, session_id=333, fold_shuffle=True)

,Description,Value
0,Session id,333
1,Target,target
2,Target type,Binary
3,Target mapping,"AbNormal: 0, Normal: 1"
4,Original data shape,"(3525, 16)"
5,Transformed data shape,"(3525, 40)"
6,Transformed train set shape,"(2467, 40)"
7,Transformed test set shape,"(1058, 40)"
8,Numeric features,3
9,Categorical features,12


In [4]:
model = compare_models(sort='F1', n_select=10)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.6676,0.6246,0.6676,0.6483,0.6520,0.1978,0.2030,0.0210
ridge,Ridge Classifier,0.6721,0.6254,0.6721,0.6493,0.6515,0.1944,0.2024,0.0200
lr,Logistic Regression,0.6761,0.6270,0.6761,0.6509,0.6500,0.1898,0.2016,0.2220
gbc,Gradient Boosting Classifier,0.6660,0.6129,0.6660,0.6444,0.6483,0.1880,0.1938,0.0920
ada,Ada Boost Classifier,0.6526,0.6071,0.6526,0.6374,0.6419,0.1789,0.1814,0.0390
lightgbm,Light Gradient Boosting Machine,0.6417,0.6032,0.6417,0.6268,0.6317,0.1563,0.1582,0.2730
rf,Random Forest Classifier,0.6312,0.6032,0.6312,0.6242,0.6267,0.1527,0.1535,0.0610
et,Extra Trees Classifier,0.6198,0.5933,0.6198,0.6184,0.6186,0.1405,0.1408,0.0590
knn,K Neighbors Classifier,0.6393,0.5798,0.6393,0.6115,0.6176,0.1165,0.1209,0.0220
dt,Decision Tree Classifier,0.6141,0.5624,0.6141,0.6117,0.6123,0.1254,0.1258,0.0200


In [5]:
tuned_model = [tune_model(i) for i in model[:9]]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6640,0.6152,0.6640,0.4409,0.5299,0.0000,0.0000
1,0.6640,0.5738,0.6640,0.4409,0.5299,0.0000,0.0000
2,0.6680,0.5273,0.6680,0.4462,0.5351,0.0000,0.0000
3,0.6680,0.5816,0.6680,0.4462,0.5351,0.0000,0.0000
4,0.6680,0.6061,0.6680,0.4462,0.5351,0.0000,0.0000
5,0.6680,0.5152,0.6680,0.4462,0.5351,0.0000,0.0000
6,0.6680,0.5289,0.6680,0.4462,0.5351,0.0000,0.0000
7,0.6667,0.5858,0.6667,0.4444,0.5333,0.0000,0.0000
8,0.6667,0.5863,0.6667,0.4444,0.5333,0.0000,0.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6964,0.6656,0.6964,0.6780,0.6766,0.2553,0.2666
1,0.6842,0.6051,0.6842,0.6595,0.6512,0.1967,0.2159
2,0.6599,0.6536,0.6599,0.6241,0.6235,0.1260,0.1386
3,0.6680,0.6476,0.6680,0.6379,0.6377,0.1586,0.1707
4,0.6680,0.5959,0.6680,0.6451,0.6489,0.1867,0.1932
5,0.6478,0.5511,0.6478,0.6146,0.6194,0.1164,0.1236
6,0.6721,0.6418,0.6721,0.6414,0.6382,0.1602,0.1751
7,0.6829,0.6330,0.6829,0.6587,0.6564,0.2041,0.2176
8,0.6789,0.6213,0.6789,0.6574,0.6594,0.2126,0.2208


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6923,0.6570,0.6923,0.6728,0.6713,0.2429,0.2545
1,0.6802,0.5964,0.6802,0.6527,0.6400,0.1724,0.1955
2,0.6721,0.6608,0.6721,0.6402,0.6355,0.1543,0.1707
3,0.6761,0.6468,0.6761,0.6508,0.6512,0.1903,0.2011
4,0.6923,0.6041,0.6923,0.6701,0.6664,0.2255,0.2404
5,0.6437,0.5452,0.6437,0.6085,0.6138,0.1032,0.1100
6,0.6640,0.6347,0.6640,0.6289,0.6265,0.1334,0.1476
7,0.6829,0.6365,0.6829,0.6578,0.6540,0.1986,0.2138
8,0.6829,0.6239,0.6829,0.6628,0.6647,0.2252,0.2331


...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6802,0.6494,0.6802,0.6567,0.6551,0.2052,0.2175
1,0.6802,0.5987,0.6802,0.6531,0.6428,0.1781,0.1991
2,0.6640,0.6399,0.6640,0.6304,0.6293,0.1394,0.1525
3,0.6761,0.6261,0.6761,0.6496,0.6489,0.1848,0.1970
4,0.6680,0.5676,0.6680,0.6422,0.6447,0.1757,0.1842
5,0.6478,0.5363,0.6478,0.6146,0.6194,0.1164,0.1236
6,0.6680,0.6510,0.6680,0.6352,0.6324,0.1468,0.1615
7,0.7033,0.6200,0.7033,0.6843,0.6773,0.2526,0.2705
8,0.6667,0.6018,0.6667,0.6439,0.6475,0.1854,0.1919


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6518,0.5764,0.6518,0.5906,0.5757,0.0432,0.0597
1,0.6599,0.6148,0.6599,0.6145,0.5982,0.0855,0.1079
2,0.6437,0.5491,0.6437,0.5781,0.5772,0.0302,0.0383
3,0.6680,0.6058,0.6680,0.6190,0.5788,0.0543,0.0875
4,0.6802,0.6045,0.6802,0.6542,0.6015,0.0993,0.1479
5,0.6397,0.5329,0.6397,0.5191,0.5400,-0.0305,-0.0507
6,0.6802,0.5868,0.6802,0.6508,0.6104,0.1129,0.1550
7,0.6911,0.6640,0.6911,0.6842,0.6174,0.1364,0.1987
8,0.6667,0.5834,0.6667,0.6199,0.5872,0.0682,0.0993


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [6]:
blended_hard = blend_models(estimator_list=tuned_model,
                       fold=10,
                       method="hard",
                       optimize="F1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6923,0.0000,0.6923,0.6728,0.6713,0.2429,0.2545
1,0.6761,0.0000,0.6761,0.6477,0.6396,0.1705,0.1894
2,0.6680,0.0000,0.6680,0.6339,0.6296,0.1408,0.1568
3,0.6721,0.0000,0.6721,0.6438,0.6433,0.1717,0.1839
4,0.6721,0.0000,0.6721,0.6478,0.6501,0.1885,0.1970
5,0.6437,0.0000,0.6437,0.6106,0.6163,0.1093,0.1156
6,0.6680,0.0000,0.6680,0.6366,0.6351,0.1528,0.1661
7,0.6870,0.0000,0.6870,0.6635,0.6596,0.2116,0.2267
8,0.6667,0.0000,0.6667,0.6439,0.6475,0.1854,0.1919


In [7]:
final_model = finalize_model(blended_hard)
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [8]:
final_model

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None, include=['13', '14', '15'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              keep_empty_features=False,
                                                              missing_values=nan,
                                                              strateg...
                                                                    monotonic_cst=None,
                                                                    n_estimators=280,
                                                                    n_jobs=-1,
                                                                    oob_score=False,
                                                                    random_state=333,
                                                                    verbose=0,
                                                                    warm_start=False)),
                                              ('K Neighbors Classifier',
                                               KNeighborsClassifier(algorithm='auto',
                                                                    leaf_size=30,
                                                                    metric='minkowski',
                                                                    metric_params=None,
                                                                    n_jobs=-1,
                                                                    n_neighbors=28,
                                                                    p=2,
                                                                    weights='uniform'))],
                                  flatten_transform=True, n_jobs=-1,
                                  verbose=False, voting='hard',
                                  weights=None))],
         verbose=False)

In [9]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "pca_test_.csv"))

train_columns = train_data.columns
#test_data = test_data[train_columns]

df_test_x = test_data#.drop(columns=['target'])

In [10]:
test_pred = predict_model(final_model, data=test_data)
test_pred

[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,prediction_label
0,Dam dispenser #2,AJX75334501,3J1XF767-1,AJX75334501,3J1XF767-1,OK,Fill1 dispenser #2,AJX75334501,3J1XF767-1,Fill2 dispenser #2,AJX75334501,3J1XF767-1,NaN,1537.304077,-8587.737305,-1290.009155,AbNormal
1,Dam dispenser #2,AJX75334501,4B1XD472-2,AJX75334501,4B1XD472-2,OK,Fill1 dispenser #2,AJX75334501,4B1XD472-2,Fill2 dispenser #2,AJX75334501,4B1XD472-2,NaN,-5808.053711,2417.458252,-613.402466,Normal
2,Dam dispenser #1,AJX75334501,3H1XE355-1,AJX75334501,3H1XE355-1,OK,Fill1 dispenser #1,AJX75334501,3H1XE355-1,Fill2 dispenser #1,AJX75334501,3H1XE355-1,NaN,12760.987305,2190.877686,3088.680664,AbNormal
3,Dam dispenser #2,AJX75334501,3L1XA128-1,AJX75334501,3L1XA128-1,OK,Fill1 dispenser #2,AJX75334501,3L1XA128-1,Fill2 dispenser #2,AJX75334501,3L1XA128-1,NaN,-12905.684570,-2384.216797,2237.460693,Normal
4,Dam dispenser #1,AJX75334501,4A1XA639-1,AJX75334501,4A1XA639-1,OK,Fill1 dispenser #1,AJX75334501,4A1XA639-1,Fill2 dispenser #1,AJX75334501,4A1XA639-1,NaN,-3197.746094,3308.233398,-1520.130127,AbNormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,AJX75334501,3K1XB597-1,OK,Fill1 dispenser #2,AJX75334501,3K1XB597-1,Fill2 dispenser #2,AJX75334501,3K1XB597-1,NaN,-12908.366211,-2379.953125,2232.846924,AbNormal
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,AJX75334501,4A1XB974-1,OK,Fill1 dispenser #2,AJX75334501,4A1XB974-1,Fill2 dispenser #2,AJX75334501,4A1XB974-1,NaN,-3385.885986,3359.517090,-1899.730835,Normal
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,AJX75334501,3L1XA998-1,OK,Fill1 dispenser #1,AJX75334501,3L1XA998-1,Fill2 dispenser #1,AJX75334501,3L1XA998-1,NaN,-2732.416504,2977.588379,-1937.937500,AbNormal
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,AJX75334501,3F1XC376-1,OK,Fill1 dispenser #1,AJX75334501,3F1XC376-1,Fill2 dispenser #1,AJX75334501,3F1XC376-1,NaN,18232.107422,-1331.921143,-2312.770508,AbNormal


In [11]:
df_sub = pd.read_csv("../data/submission.csv")
df_sub["target"] = test_pred["prediction_label"]
df_sub.to_csv("../data/automl/submission-18.csv", index=False)